**`Compare runs; using hyperparameter tuning, Choose a model & Deploy it to a REST API`**

In this section, the following will be carried out;

- Run a hyperparameter sweep using `hyperopt`.
- Compare results of the runs in the MLflow UI.
- Choose the best run and register it as a model.
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud environment.

In [1]:
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)

import keras
import numpy as np
import pandas as pd
import mlflow

from mlflow.models import infer_signature
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [2]:
""" RUN THE COMMAND BELOW IN THE TERMINAL. """
# export MLFOW_TRACKING_URI=http://localhost:5000

' RUN THE COMMAND BELOW IN THE TERMINAL. '

Now load the Dataset and split into it into training, validation and test sets.

In [3]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep = ";"
)

In [ ]:
train, test = train_test_split(data, test_size = 0.25, random_state = 42)
train_X = train.drop(["quality"], axis = 1).values
train_y = train[["quality"]].values.ravel()
test_X = test.drop(["quality"], axis = 1).values
test_y = test[["quality"]].values.ravel()

train_X, validation_X, train_y, validation_y = train_test_split(
    train_X, train_y, test_size = 0.2, random_state = 42
)

In [ ]:
# Infer madel signature.

signature = infer_signature(train_X, train_y)